<a href="https://colab.research.google.com/github/balu1866/TeluguLLM/blob/main/Telugu_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install Langchain
%pip install datasets
%pip install -U bitsandbytes

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\balup\Github\TeluguLLM\.teluguLLM\Scripts\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\balup\Github\TeluguLLM\.teluguLLM\Scripts\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\balup\Github\TeluguLLM\.teluguLLM\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import random

df = pd.read_csv("hf://datasets/Telugu-LLM-Labs/telugu_alpaca_yahma_cleaned_filtered_romanized/yahma_alpaca_cleaned_telugu_filtered_and_romanized.csv")

c:\Users\balup\Github\TeluguLLM\.teluguLLM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = df[['instruction', 'output', 'telugu_instruction', 'telugu_output', 'telugu_transliterated_instruction', 'telugu_transliterated_output']]

In [5]:

import json

eng_tel_map = {}
eng_tel_list =[]

eng_tel_trans_map = {}
eng_tel_trans_list =[]

tel_trans_eng_map = {}
tel_trans_eng_list =[]

tel_trans_tel_map = {}
tel_trans_tel_list =[]

tel_trans_tel_trans_map = {}
tel_trans_tel_trans_list =[]

tel_tel_trans_map = {}
tel_tel_trans_list =[]

tel_eng_map = {}
tel_eng_list =[]

tel_tel_map = {}
tel_tel_list =[]


for index, row in df.iterrows():
  # Access individual columns using their names
  instruction = row['instruction']
  output = row['output']
  telugu_instruction = row['telugu_instruction']
  telugu_output = row['telugu_output']
  telugu_transliterated_instruction = row['telugu_transliterated_instruction']
  telugu_transliterated_output = row['telugu_transliterated_output']

  # Telugu output
  eng_tel_list.append({
      'english_instruction': instruction,
      'telugu_output': telugu_output,
  })

  tel_tel_list.append({
      'telugu_instruction': telugu_instruction,
      'telugu_output': telugu_output,
  })

  tel_trans_tel_list.append({
      'telugu_transliterated_instruction': telugu_transliterated_instruction,
      'telugu_output': telugu_output,
  })

  # Telugu transliterated output
  eng_tel_trans_list.append({
      'english_instruction': instruction,
      'telugu_transliterated_output': telugu_transliterated_output,
  })

  tel_tel_trans_list.append({
      'telugu_instruction': telugu_instruction,
      'telugu_transliterated_output': telugu_transliterated_output,
  })

  tel_trans_tel_trans_list.append({
      'telugu_transliterated_instruction': telugu_transliterated_instruction,
      'telugu_transliterated_output': telugu_transliterated_output,
  })

  # English output
  tel_eng_list.append({
      'telugu_instruction': telugu_instruction,
      'english_output': output,
  })

  tel_trans_eng_list.append({
      'telugu_transliterated_instruction': telugu_transliterated_instruction,
      'english_output': output,
  })


dataset_list = []
dataset_list.extend(eng_tel_list)
dataset_list.extend(tel_tel_list)
dataset_list.extend(tel_trans_tel_list)

dataset_list.extend(eng_tel_trans_list)
dataset_list.extend(tel_tel_trans_list)
dataset_list.extend(tel_trans_tel_trans_list)

dataset_list.extend(tel_eng_list)
dataset_list.extend(tel_trans_eng_list)

random.shuffle(dataset_list)


In [6]:
def format_record(example):
    if 'english_instruction' in example and 'telugu_output' in example:
        return {
            "prompt": f"Translate the following instruction to Telugu:\n{example['english_instruction']}",
            "completion": example["telugu_output"]
        }

    elif 'telugu_instruction' in example and 'telugu_output' in example:
        return {
            "prompt": f"Rewrite the following in Telugu:\n{example['telugu_instruction']}",
            "completion": example["telugu_output"]
        }

    elif 'telugu_transliterated_instruction' in example and 'telugu_output' in example:
        return {
            "prompt": f"Translate this transliterated instruction to Telugu:\n{example['telugu_transliterated_instruction']}",
            "completion": example["telugu_output"]
        }

    elif 'english_instruction' in example and 'telugu_transliterated_output' in example:
        return {
            "prompt": f"Translate to Telugu (transliterated):\n{example['english_instruction']}",
            "completion": example["telugu_transliterated_output"]
        }

    elif 'telugu_instruction' in example and 'telugu_transliterated_output' in example:
        return {
            "prompt": f"Transliterate the following Telugu instruction:\n{example['telugu_instruction']}",
            "completion": example["telugu_transliterated_output"]
        }


    elif 'telugu_transliterated_instruction' in example and 'telugu_transliterated_output' in example:
        return {
            "prompt": f"Complete this transliterated Telugu instruction:\n{example['telugu_transliterated_instruction']}",
            "completion": example["telugu_transliterated_output"]
        }

    elif 'telugu_instruction' in example and 'english_output' in example:
        return {
            "prompt": f"Translate this Telugu instruction to English:\n{example['telugu_instruction']}",
            "completion": example["english_output"]
        }

    elif 'telugu_transliterated_instruction' in example and 'english_output' in example:
        return {
            "prompt": f"Translate this transliterated Telugu instruction to English:\n{example['telugu_transliterated_instruction']}",
            "completion": example["english_output"]
        }

    else:
        return None

formatted_data = []

for item in dataset_list:
    formatted = format_record(item)
    if formatted:  # skip malformed
        formatted_data.append(formatted)

with open('dataset.json', 'w', encoding='utf-8') as file:
  for item in formatted_data:
    file.write(json.dumps(item) + '\n')



In [7]:
%huggingface-cli whoami

UsageError: Line magic function `%huggingface-cli` not found.


In [7]:
import torch 
print(torch.cuda.is_available())

True


In [3]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126


Looking in indexes: https://download.pytorch.org/whl/cu126, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.



In [4]:
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token



model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    load_in_8bit=True, 
    llm_int8_enable_fp32_cpu_offload=True)


model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, peft_config)



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [00:25<00:00,  8.41s/it]


In [12]:
from datasets import load_dataset, Dataset

def tokenize(record):
  full_prompt = record['prompt'] + "\n" + record['completion']
  tokenized = tokenizer(
      full_prompt,
      max_length=512,
      padding="max_length",
      truncation=True,
  )
  prompt_ids = tokenizer(record['prompt'], truncation=True, max_length=512)['input_ids']
  prompt_ids_len = len(prompt_ids)

  input_ids = tokenized['input_ids']
  # if not isinstance(input_ids,  list):
  #   input_ids = input_ids.tolist()
  labels = input_ids.copy()  # make a copy of input_ids for labels
  labels[:prompt_ids_len] = [-100] * prompt_ids_len
  tokenized['labels'] = labels
  return tokenized

# dataset = Dataset.from_list(formatted_data)
dataset = load_dataset("json", data_files="dataset.json")

split_dataset = dataset["train"].train_test_split(test_size=0.2)

train_dataset = split_dataset["train"]
temp_dataset = split_dataset["test"].train_test_split(test_size=0.5)

val_dataset = temp_dataset["train"]
test_dataset = temp_dataset["test"]

tokenized_train = train_dataset.map(tokenize, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(tokenize, remove_columns=val_dataset.column_names)
tokenized_test = test_dataset.map(tokenize, remove_columns=test_dataset.column_names)


print(tokenized_train[0])


Map: 100%|██████████| 23128/23128 [00:45<00:00, 504.86 examples/s]

{'input_ids': [1, 5103, 25245, 1148, 1040, 3064, 19126, 26981, 13894, 29515, 781, 995, 947, 917, 32753, 29473, 32442, 995, 947, 922, 31958, 32316, 32616, 995, 948, 910, 29473, 995, 947, 945, 31958, 31995, 32442, 31633, 32442, 31995, 29473, 995, 947, 943, 995, 948, 907, 32753, 31633, 29473, 32616, 995, 948, 906, 32442, 31633, 32616, 32103, 29473, 995, 947, 908, 32316, 995, 947, 932, 32050, 32616, 32442, 31995, 29473, 995, 947, 955, 995, 948, 901, 995, 947, 925, 31995, 995, 947, 955, 31633, 32447, 995, 948, 901, 29473, 995, 947, 955, 32316, 995, 947, 941, 32050, 995, 947, 937, 32753, 995, 948, 899, 995, 947, 946, 32316, 29473, 31958, 32050, 995, 947, 946, 32316, 995, 947, 932, 31995, 29491, 781, 3272, 5595, 24307, 1205, 2944, 15604, 29493, 11722, 29484, 21134, 11412, 4675, 3794, 1176, 1756, 8001, 29493, 13834, 29536, 5875, 1032, 1079, 2263, 15604, 1408, 29474, 26501, 1051, 15822, 1057, 2139, 1089, 1106, 3718, 1857, 29478, 29491, 9233, 29488, 1322, 29486, 1085, 26501, 1530, 1296, 29474, 6

In [13]:
tokenized_train.save_to_disk("tokenized_data/tokenized_train_dataset")
tokenized_val.save_to_disk("tokenized_data/tokenized_val_dataset")
tokenized_test.save_to_disk("tokenized_data/tokenized_test_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 23128/23128 [00:00<00:00, 51211.72 examples/s]


In [5]:
from datasets import load_from_disk
tokenized_train = load_from_disk("tokenized_data/tokenized_train_dataset")
tokenized_val=load_from_disk("tokenized_data/tokenized_val_dataset")
tokenized_test=load_from_disk("tokenized_data/tokenized_test_dataset")

In [9]:
import torch
print(torch.cuda.is_available())

True


In [6]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./lora-mistral-telugu",  # Directory to save the model and checkpoints
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch   
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    save_strategy="epoch",  # Save the model at the end of each epoch
    logging_steps=10,  # Log every 10 steps
    learning_rate=2e-4,  # Learning rate for the optimizer
    weight_decay=0.01,  # Weight decay for regularization
    push_to_hub=False,
    report_to="none",
    label_names=["labels"],
    fp16=True,  # Use mixed precision training (if supported by your hardware)
    load_best_model_at_end=True,  # Load the best model at the end of training
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to False for causal language modeling
)
trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    args = training_args,
    train_dataset = tokenized_val,
    eval_dataset = tokenized_val,
    data_collator = data_collator 
)

print(model.hf_device_map)


trainer.train() 
print(model.hf_device_map)
results = trainer.evaluate()  # Evaluate the model on the validation set
print(results)  # Print the evaluation results

c:\Users\balup\Github\TeluguLLM\.teluguLLM\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\balup\AppData\Local\Temp\ipykernel_1056\2565005347.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


{'': 0}


c:\Users\balup\Github\TeluguLLM\.teluguLLM\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\balup\Github\TeluguLLM\.teluguLLM\lib\site-packages\bitsandbytes\autograd\_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


KeyboardInterrupt: 